In [1]:
!sudo pip install -q transformers --upgrade

In [2]:
!sudo pip install -q autoawq

In [4]:
import torch

In [5]:
new_model_name = "/data/mistral/query-to-mql/exp-10/nov-20/checkpoint-8000"
torch.cuda.empty_cache()

from peft import AutoPeftModelForCausalLM

model = AutoPeftModelForCausalLM.from_pretrained(new_model_name, device_map="auto", torch_dtype=torch.bfloat16)
model = model.merge_and_unload()

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [ ]:
# from transformers import AutoModelForCausalLM, AutoTokenizer
# model = AutoModelForCausalLM.from_pretrained(model)

In [6]:
# Load LLaMA tokenizer
from transformers import AutoModelForCausalLM, AutoTokenizer
model_name = "mistralai/Mistral-7B-Instruct-v0.1"
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True,
                                          # add_eos_token=True,
                                          use_fast=False)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

NameError: name 'AutoTokenizer' is not defined

In [ ]:
from transformers import BitsAndBytesConfig
    bnb_config = BitsAndBytesConfig(
        load_in_4bit=True,
        bnb_4bit_quant_type="nf4",
        bnb_4bit_use_double_quant=True,
    )

In [ ]:
from autoawq import AWQQuantizer

quantizer = AWQQuantizer(
    model=model,
    tokenizer=tokenizer,
    bnb_config=bnb_config,
    quant_method="awq",
    num_epochs=10,
    lr=1e-4,
    batch_size=1,
    eval_batch_size=1,
    max_length=2048,
    device="cuda",
)

In [ ]:
from datasets import load_dataset
dataset = load_dataset("wikitext", "wikitext-2-raw-v1")


In [ ]:
quantizer.quantize(
    input_data=dataset["train"],
    valid_data=dataset["validation"],
)